In [1]:
# Import the libraries
import geemap
import ee
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import geopandas as gpd

In [2]:
# Authenticate your GEE account
ee.Authenticate()

True

In [3]:
# Initialize GEE 
ee.Initialize()

In [4]:
# Add a map to display the image and index
Map = geemap.Map(center = (12.20, -16.36), zoom = 12)
Map

Map(center=[12.2, -16.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [5]:
# Asegurarnos de la creación de una geometría
geometry = ee.Geometry.Polygon([[[-16.458035, 12.233829],
   [-16.421471, 12.233829],
   [-16.421471, 12.257315],
   [-16.458035, 12.257315],
   [-16.458035, 12.233829]]])

In [13]:
# Definir la colección de imágenes y el área de interés
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Imagenes_planet_folder/SAR_collection") \
              .filterDate('2022-07-01', '2024-01-30') \
              .filterBounds(geometry)

In [14]:
import geopandas as gpd

# Cargar el shapefile
shapefile_path = '../DataIn/Parcelas_poligono.shp'
gdf = gpd.read_file(shapefile_path)

In [15]:
import geopandas as gpd
import json
import ee
import geemap

# Inicializa Earth Engine
ee.Initialize()


# Cargar el shapefile
shapefile_path = '../DataIn/Parcelas_poligono.shp'
gdf = gpd.read_file(shapefile_path)

# Convertir el GeoDataFrame a GeoJSON
geojson = json.loads(gdf.to_json())

# Convertir el GeoJSON a ee.FeatureCollection
def geojson_to_ee(geojson):
    features = []
    for feature in geojson['features']:
        geom = feature['geometry']
        props = feature['properties']
        ee_geom = ee.Geometry(geom)
        ee_feature = ee.Feature(ee_geom, props)
        features.append(ee_feature)
    return ee.FeatureCollection(features)

fc = geojson_to_ee(geojson)

print(fc.getInfo())  # Verifica la conversión

{'type': 'FeatureCollection', 'columns': {'area_ha': 'Float', 'area_m2': 'Float', 'location_i': 'String', 'polygon_id': 'Integer', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-16.45194989308384, 12.245266561121108], [-16.451980836287003, 12.24523403971215], [-16.45199601597157, 12.24521578207729], [-16.45200243814581, 12.245201518299176], [-16.452001270477773, 12.245188966173801], [-16.45197733328287, 12.245168996882203], [-16.451946973913728, 12.245141610422696], [-16.45193529723329, 12.24513134049965], [-16.451927707391004, 12.245122782230135], [-16.451924788220897, 12.245116506165651], [-16.4519142792085, 12.24513134049965], [-16.45190552169817, 12.245142180973966], [-16.451897348021863, 12.245148457037839], [-16.451889758179576, 12.245154733101566], [-16.45188158450327, 12.24516100916514], [-16.451875162329028, 12.245169567433413], [-16.451867572486744, 12.245174131843044], [-16.451867572486744, 12.245179837354975], 

In [16]:
import pandas as pd

# Extraer datos para cada polígono y cada imagen
def extract_data(image):
    def zonal_stats(feature):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=3,  #
            maxPixels=1e9
        )
        return feature.set(stats).set({'image_date': image.date().format('YYYY-MM-dd')})
    
    return fc.map(zonal_stats)

# Iterar sobre la colección y extraer datos
results = []
image_list = collection.toList(collection.size())

for i in range(collection.size().getInfo()):
    image = ee.Image(image_list.get(i))
    image = image # anny operation/index should be defined in a previous functions like addindex() and mapped here to be included in the calculations
    data = extract_data(image).getInfo()
    
    for feature in data['features']:
        properties = feature['properties']
        properties['geometry'] = feature['geometry']
        results.append(properties)

In [17]:
# Convertir los resultados a un DataFrame de pandas
df = pd.DataFrame(results)

# Exportar a un archivo CSV
df.to_csv('../Results/output_indices_sentinel_1.csv', index=False)

print('Exportación completada.')

Exportación completada.
